In [1]:
import numpy as np
import pandas as pd
import os
import pmdarima as pm
from pmdarima import pipeline
from pmdarima import model_selection
from pmdarima import preprocessing as ppc
from pmdarima import arima
from matplotlib import pyplot as plt

In [2]:
def scale(x):
    return (x - x.mean())/x.std()

def winsor(x, floor, root): 
    x[x < x.quantile(floor)] = x.quantile(floor)
    x[x > x.quantile(root)] = x.quantile(root)
    return x

def type_change(x):
    res = []
    for i in x:
        try:
            res.append(float(i))
        except (ValueError):
            res.append(np.nan)
    return res

In [68]:
## 数据平稳处理

def station_process(x):
    n_diff = pm.arima.ndiffs(x)
    if n_diff == 0:
        res = x
    elif n_diff == 1:
        res = x.diff()
        res.name = res.name + "_diff1"
    else:
        res = x.diff().diff()
        res.name = res.name + "_diff2"
    return res

## 样本划分
def sample_split(x,val_date,test_date):
    # val_date = pd.to_datetime(val_date)
    # test_date = pd.to_datetime(val_date)
    train_x = x.truncate(after=val_date)
    val_x = x.truncate(before = val_date, after=test_date)
    test_x = x.truncate(before = test_date)
    return train_x, val_x, test_x

In [3]:
WindA = pd.read_excel('wind全A行情.xlsx', parse_dates = True)
WindA = WindA.iloc[:WindA.shape[0]-2,:]
WindA.columns = list(map(lambda x: x.split(sep="\n")[0],WindA.columns.tolist()))
WindA.set_index("时间",inplace=True)

In [4]:
WindA_v1 = pd.DataFrame()
WindA_v1["涨跌幅"] = (WindA["收盘价"] - WindA["前收盘价"])/WindA["前收盘价"]
WindA_v1[['收盘价','成交量', '换手率', '周最高价',
       '周最低价', '周均价', '周成交量', '周涨跌幅', '自由流通股本', '市盈率PE(TTM)',
       '市净率PB(LF)', '股息率(近12个月)', '一致预测净利润(FY1)', '一致预测净利润(FY2)']] = WindA[['收盘价','成交量', '换手率', '周最高价',
       '周最低价', '周均价', '周成交量', '周涨跌幅', '自由流通股本', '市盈率PE(TTM)',
       '市净率PB(LF)', '股息率(近12个月)', '一致预测净利润(FY1)', '一致预测净利润(FY2)']]
# WindA_v1 = WindA_v1.apply(winsor,floor=0.025,root=0.975,axis = 0)
# WindA_v1 = WindA_v1.apply(scale,axis=0)
WindA_v1.columns = "WindA_" + WindA_v1.columns

## 工业行情（稳定）

In [20]:
Industry = pd.read_excel('工业行情.xlsx', parse_dates = True)
Industry = Industry.iloc[:Industry.shape[0]-2,:]
Industry.columns = list(map(lambda x: x.split(sep="\n")[0],Industry.columns.tolist()))
Industry.set_index("时间",inplace=True)

In [21]:
Industry_v1 = pd.DataFrame()
Industry_v1["涨跌幅"] = (Industry["收盘价"] - Industry["前收盘价"])/Industry["前收盘价"]
Industry_v1[['收盘价','成交量', '换手率', '周最高价',
       '周最低价', '周均价', '周成交量', '周涨跌幅', '自由流通股本', '市盈率PE(TTM)',
       '市净率PB(LF)', '股息率(近12个月)', '一致预测净利润(FY1)', '一致预测净利润(FY2)']] = Industry[['收盘价','成交量', '换手率', '周最高价',
       '周最低价', '周均价', '周成交量', '周涨跌幅', '自由流通股本', '市盈率PE(TTM)',
       '市净率PB(LF)', '股息率(近12个月)', '一致预测净利润(FY1)', '一致预测净利润(FY2)']]
# Industry_v1 = Industry_v1.apply(winsor,floor=0.025,root=0.975,axis = 0)
# Industry_v1 = Industry_v1.apply(scale,axis=0)
Industry_v1.columns = "Industry_" + Industry_v1.columns

## 信息技术(成长)

In [22]:
info_tech = pd.read_excel('工业行情.xlsx', parse_dates = True)
info_tech = info_tech.iloc[:info_tech.shape[0]-2,:]
info_tech.columns = list(map(lambda x: x.split(sep="\n")[0],info_tech.columns.tolist()))
info_tech.set_index("时间",inplace=True)

In [23]:
info_tech_v1 = pd.DataFrame()
info_tech_v1["涨跌幅"] = (info_tech["收盘价"] - info_tech["前收盘价"])/info_tech["前收盘价"]
info_tech_v1[['收盘价','成交量', '换手率', '周最高价',
       '周最低价', '周均价', '周成交量', '周涨跌幅', '自由流通股本', '市盈率PE(TTM)',
       '市净率PB(LF)', '股息率(近12个月)', '一致预测净利润(FY1)', '一致预测净利润(FY2)']] = info_tech[['收盘价','成交量', '换手率', '周最高价',
       '周最低价', '周均价', '周成交量', '周涨跌幅', '自由流通股本', '市盈率PE(TTM)',
       '市净率PB(LF)', '股息率(近12个月)', '一致预测净利润(FY1)', '一致预测净利润(FY2)']]
# info_tech_v1 = info_tech_v1.apply(winsor,floor=0.025,root=0.975,axis = 0)
# info_tech_v1 = info_tech_v1.apply(scale,axis=0)
info_tech_v1.columns = "info_tech_" + info_tech_v1.columns

In [34]:
windA_info_tech_v1 = pd.merge(WindA_v1,info_tech_v1,left_index=True,right_index=True)
windA_Industry_v1 = pd.merge(WindA_v1,Industry_v1,left_index=True,right_index=True)

## 宏观变量

In [35]:
macro_var = pd.read_excel('宏观变量.xlsx', parse_dates = True)

macro_var.columns = ['时间', 'CPI', 'PPI', '出口价格指数(总)',
       '出口价格指数(车船航)', '进口价格指数(总)',
       '进口价格指数(车船航)', '固定资产投资完成额', '社会消费品零售总额',
       'M2', '金融机构各项贷款余额', '社会融资规模']
       
macro_var.set_index('时间',inplace=True)
macro_var.index = macro_var.index.strftime("%Y-%m")
macro_var.columns = "宏观_" + macro_var.columns

In [36]:
temp = windA_info_tech_v1
temp["date_trans"] = windA_info_tech_v1.index.strftime("%Y-%m")
info_macro = pd.merge(temp,macro_var,left_on="date_trans",right_index=True)
del info_macro["date_trans"]

In [37]:
temp = windA_Industry_v1
temp["date_trans"] = windA_Industry_v1.index.strftime("%Y-%m")
industry_macro = pd.merge(temp,macro_var,left_on="date_trans",right_index=True)
del industry_macro["date_trans"]

In [38]:
info_macro.to_excel("data_prepocessed/信息行业和宏观_unscale.xlsx")
industry_macro.to_excel("data_prepocessed/工业行业和宏观_unscale.xlsx")

In [39]:
## 数据平稳化处理
temp = pd.DataFrame()
for i in info_macro.columns:
    temp = pd.concat([temp, station_process(info_macro[i])],axis=1)
    train_temp, val_temp, test_temp = sample_split(temp,val_date="2021-01-01",test_date="2022-01-01")
    

### 个股

#### 成长型

In [101]:

## 宏观数据平稳化处理
macro_temp = pd.DataFrame()
for i in info_macro.columns:
    macro_temp = pd.concat([macro_temp, station_process(info_macro[i])],axis=1)
train_macro_temp, val_macro_temp, test_macro_temp = sample_split(macro_temp,val_date="2021-01-01",test_date="2022-01-01")
train_macro_temp = train_macro_temp.apply(winsor,floor=0.025,root=0.975,axis = 0)
train_macro_temp= train_macro_temp.apply(scale,axis=0)
val_macro_temp = val_macro_temp.apply(winsor,floor=0.025,root=0.975,axis = 0)
val_macro_temp= val_macro_temp.apply(scale,axis=0)
test_macro_temp = test_macro_temp.apply(winsor,floor=0.025,root=0.975,axis = 0)
test_macro_temp= test_macro_temp.apply(scale,axis=0)
macro_temp = pd.concat([train_macro_temp,val_macro_temp,test_macro_temp],axis=0)

## 成长
for i, stk in enumerate(os.listdir("成长")):
    temp = pd.read_excel("成长/"+stk,parse_dates=True)
    temp = temp.iloc[:temp.shape[0]-2,:]
    temp.columns = list(map(lambda x: x.split(sep="\n")[0],temp.columns.tolist()))
    temp.set_index("日期",inplace=True)
    stk_name = temp["简称"][-1]
    temp.drop(temp.loc[:,"代码":"简称"].columns, axis = 1, inplace=True)
    del temp["B股流通市值(元)"]
    ## 因变脸
    ## 未来一期价格
    Y = pd.Series(temp["收盘价(元)"][1:].tolist(), index = list(temp.index)[:-1],name="股价")

    ## 类型转化
    for column in temp.columns:
        if temp[column].dtype == object:
            temp[column] = type_change(temp[column])
        temp[column].astype(float)

    ## 缺失值填充
    if temp.isnull().sum().sum() == 0:
        pass
    else:
        temp = temp.interpolate()

    ## 判断平稳性，不同数据集不同标准化 
    stk_temp = pd.DataFrame()
    for j in temp.columns:
        stk_temp = pd.concat([stk_temp, station_process(temp[j])],axis=1)
    train_stk_temp, val_stk_temp, test_stk_temp = sample_split(stk_temp,val_date="2021-01-01",test_date="2022-01-01")
    train_stk_temp = train_stk_temp.apply(winsor,floor=0.025,root=0.975,axis = 0)
    train_stk_temp= train_stk_temp.apply(scale,axis=0)
    val_stk_temp = val_stk_temp.apply(winsor,floor=0.025,root=0.975,axis = 0)
    val_stk_temp= val_stk_temp.apply(scale,axis=0)
    test_stk_temp = test_stk_temp.apply(winsor,floor=0.025,root=0.975,axis = 0)
    test_stk_temp= test_stk_temp.apply(scale,axis=0)
    stk_temp = pd.concat([train_stk_temp,val_stk_temp,test_stk_temp],axis=0)

    temp = pd.merge(Y,stk_temp,left_index=True,right_index=True,how="left")
    res = pd.merge(temp, macro_temp,left_index=True,right_index=True,how="left").fillna(method="ffill")
    res = res.dropna(axis = 0)
    res.to_excel("data_new/"+stk_name+"_"+stk[:6]+"_成长.xlsx")

#### 稳定

In [104]:
## 宏观数据平稳化处理
macro_temp = pd.DataFrame()
for i in industry_macro.columns:
    macro_temp = pd.concat([macro_temp, station_process(industry_macro[i])],axis=1)
train_macro_temp, val_macro_temp, test_macro_temp = sample_split(macro_temp,val_date="2021-01-01",test_date="2022-01-01")
train_macro_temp = train_macro_temp.apply(winsor,floor=0.025,root=0.975,axis = 0)
train_macro_temp= train_macro_temp.apply(scale,axis=0)
val_macro_temp = val_macro_temp.apply(winsor,floor=0.025,root=0.975,axis = 0)
val_macro_temp= val_macro_temp.apply(scale,axis=0)
test_macro_temp = test_macro_temp.apply(winsor,floor=0.025,root=0.975,axis = 0)
test_macro_temp= test_macro_temp.apply(scale,axis=0)
macro_temp = pd.concat([train_macro_temp,val_macro_temp,test_macro_temp],axis=0)

## 稳定
for i, stk in enumerate(os.listdir("稳定")):
    temp = pd.read_excel("稳定/"+stk,parse_dates=True)
    temp = temp.iloc[:temp.shape[0]-2,:]
    temp.columns = list(map(lambda x: x.split(sep="\n")[0],temp.columns.tolist()))
    temp.set_index("日期",inplace=True)
    stk_name = temp["简称"][-1]
    temp.drop(temp.loc[:,"代码":"简称"].columns, axis = 1, inplace=True)
    del temp["B股流通市值(元)"]
    ## 因变脸
    ## 未来一期价格
    Y = pd.Series(temp["收盘价(元)"][1:].tolist(), index = list(temp.index)[:-1],name="股价")

    ## 类型转化
    for column in temp.columns:
        if temp[column].dtype == object:
            temp[column] = type_change(temp[column])
        temp[column].astype(float)

    ## 缺失值填充
    if temp.isnull().sum().sum() == 0:
        pass
    else:
        temp = temp.interpolate()
    
    ## 判断平稳性，不同数据集不同标准化 
    stk_temp = pd.DataFrame()
    for j in temp.columns:
        stk_temp = pd.concat([stk_temp, station_process(temp[j])],axis=1)
    train_stk_temp, val_stk_temp, test_stk_temp = sample_split(stk_temp,val_date="2021-01-01",test_date="2022-01-01")
    train_stk_temp = train_stk_temp.apply(winsor,floor=0.025,root=0.975,axis = 0)
    train_stk_temp= train_stk_temp.apply(scale,axis=0)
    val_stk_temp = val_stk_temp.apply(winsor,floor=0.025,root=0.975,axis = 0)
    val_stk_temp= val_stk_temp.apply(scale,axis=0)
    test_stk_temp = test_stk_temp.apply(winsor,floor=0.025,root=0.975,axis = 0)
    test_stk_temp= test_stk_temp.apply(scale,axis=0)
    stk_temp = pd.concat([train_stk_temp,val_stk_temp,test_stk_temp],axis=0)

    temp = temp.apply(winsor,floor=0.025,root=0.975,axis = 0)
    temp= temp.apply(scale,axis=0)

    temp = pd.merge(Y,stk_temp,left_index=True,right_index=True,how="left")
    res = pd.merge(temp, macro_temp,left_index=True,right_index=True,how="left").fillna(method="ffill")
    res = res.dropna(axis = 0)
    res.to_excel("data_new/"+stk_name+"_"+stk[:6]+"_稳定.xlsx")